# Tarea 1 - Introducción a Data Science

#### Integrantes:
Axel Mondaca

Sebastián Hernández

### Instalación de librerías

In [ ]:
%pip install selenium
%pip install pandas
%pip install matplotlib

### Importar librerías

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import matplotlib.pyplot as plt
import math
import re

### Código

In [ ]:
base_url = "https://nextspaceflight.com/launches/past/" #Indicamos el sitio web
last_page = 240 #Indicamos el total de páginas

In [ ]:
options = Options()
options.add_argument("--headless")
browser = webdriver.Chrome(options=options)
browser.get(base_url)

browser2 = webdriver.Chrome(options=options)
#Obtención de datos
launch_data = []
for page in range(1, 2):  
    browser.get(f"{base_url}?page={page}") # obtenemos el html (hace la petición), sin selenium sería la librería de requests
    cards = browser.find_elements(By.CLASS_NAME, "mdl-card")
    averageprice = 0
    averageheight = 0
    Payloadweightaverage = 0
    pricecounter = 0
    heightcounter = 0
    payloadweightcounter = 0
    for card in cards:
        company = card.find_element(By.CLASS_NAME, "mdl-card__title-text").text
        rocket_and_payload = card.find_element(By.CLASS_NAME, "header-style").text
        date_and_location = card.find_element(By.CLASS_NAME, "mdl-card__supporting-text").text.replace(",", " ")
        details_url = card.find_element(By.CLASS_NAME, "mdc-button").get_attribute("href")
        total_payload = 0
        rocket_height = 0
        fairing_diameter = 0
        fairing_height = 0
        price = 0

        pricecounter += 1
        heightcounter += 1
        payloadweightcounter += 1
        
        
        #Navegamos a la página de detalles
        browser2.get(details_url)
        
        try: 
            sections = browser2.find_elements(By.CLASS_NAME, "mdl-card__supporting-text")
            for section in sections:
                elements = section.find_elements(By.CLASS_NAME, "mdl-cell")
                for element in elements:
                    #Calculamos el total payload
                    text = element.text.strip()
                    if "Payload to LEO" in text:
                        number = int(text.split(":")[1].replace(",", "").replace(" kg", "").strip())
                        total_payload += number
                        Payloadweightaverage += total_payload
                        if total_payload == 0.0:
                            payloadweightcounter -= 1
                    elif "Payload to GTO" in text:
                        number = int(text.split(":")[1].replace(",", "").replace(" kg", "").strip())
                        total_payload += number
                        Payloadweightaverage += total_payload
                        if total_payload == 0.0:
                            payloadweightcounter -= 1
                    #Obtenemos las dimensiones del cohete
                    elif "Rocket Height" in text:
                        number = float(text.split(":")[1].replace("m", "").strip())
                        rocket_height += number
                    elif "Fairing Diameter" in text:
                        number = float(text.split(":")[1].replace("m", "").strip())
                        fairing_diameter += number
                    elif "Fairing Height" in text:
                        number = float(text.split(":")[1].replace("m", "").strip())
                        fairing_height += number
                        averageheight += fairing_height
                        if fairing_height == 0.0:
                            heightcounter -= 1
                    #Obtenermos el precio
                    elif "Price" in text:
                        number = float(text.split(":")[1].replace(",", "").replace("$", "").replace("million", "").strip())
                        price += number
                        averageprice += price
                        if price == 0.0:
                            pricecounter -= 1

        except Exception as e:
            print(f"Error al obtener detalles de la URL {details_url}: {e}")
        
        # Creamos el diccionario con los datos obtenidos
        launch_dict = {'company': company, 
                    'rocket_and_payload': rocket_and_payload, 
                    'date_and_location': date_and_location, 
                    'details_url': details_url,
                    'total_payload': total_payload,
                    'fairing_diameter': fairing_diameter,
                    'fairing_height': fairing_height,
                    'rocket_height': rocket_height,
                    'price': price}
    
        launch_data.append(launch_dict)
    
    averageheight = averageheight/heightcounter
    averageprice = averageprice/pricecounter
    Payloadweightaverage = Payloadweightaverage(payloadweightcounter)

    for each in launch_data:
        if total_payload == 0.0:
            total_payload = Payloadweightaverage
        if fairing_height == 0.0:
            fairing_height = averageheight
        if price == 0.0:
            price = averageprice
        volume = ((1/3)*math.pi*((fairing_diameter/2)**2)*fairing_height)+(math.pi*((fairing_diameter/2)**2)*rocket_height)
        launch_dict2 = {'volume': volume}
        launch_data.append(launch_dict2)


browser.quit()
browser2.quit()

# Crear DataFrame
df = pd.DataFrame(launch_data, columns=['company', 'rocket_and_payload', 'date_and_location', 'details_url',
                                        'total_payload', 'fairing_diameter', 'fairing_height', 'rocket_height',
                                        'price','volume'])
df.size
df.shape
df["rocket"] = df["rocket_and_payload"].str.split("|").str[0]
df["payload"] = df["rocket_and_payload"].str.split("|").str[1]
df["date"] = df["date_and_location"].str.split("\n").str[0]
df["location"] = df["date_and_location"].str.split("\n").str[1]
df.drop(columns=["rocket_and_payload", "date_and_location"], inplace=True)

df
# Exportar a CSV
df.to_csv('launch_data.csv', index=False)
